<a href="https://colab.research.google.com/github/AndreyLFR/test/blob/main/NN_voice_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Задача распознования голоса.
Найденный датасет содержит голосовые записи и информацию о гендере говорящего.
Сама история с гендером в жизни применить не получится. Однако идентификация по голосу может быть полезной.
Мы до сих пор спрашиваем, когда стучатся в дверь: Кто пришел?
Здесь система сможет идентифицировать звонящего или отвечающего

Пока тренируемся с учебном датасетом

In [112]:
from zipfile import ZipFile
import pandas as pd
import keras
from keras import layers
from keras.layers import StringLookup
from tensorflow import data as tf_data
import numpy as np
import math
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from keras import callbacks
from tensorflow.keras.layers import Dropout

In [97]:
with ZipFile("/content/sample_data/voice.csv.zip", "r") as myzip:
    myzip.extractall(path='VOICES')

In [98]:
df = pd.read_csv('/content/VOICES/voice.csv')

print(f'Наименования столбцов: {list(df)}')
print(f'Количество атрибутов: {df.shape[1]-1}') #-1 убран target
print(f'Количество записей: {df.shape[0]}')

Наименования столбцов: ['meanfreq', 'sd', 'median', 'Q25', 'Q75', 'IQR', 'skew', 'kurt', 'sp.ent', 'sfm', 'mode', 'centroid', 'meanfun', 'minfun', 'maxfun', 'meandom', 'mindom', 'maxdom', 'dfrange', 'modindx', 'label']
Количество атрибутов: 20
Количество записей: 3168


Target у нас представлен значениями

In [99]:
print(df['label'].unique())

['male' 'female']


In [100]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3168 entries, 0 to 3167
Data columns (total 21 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   meanfreq  3168 non-null   float64
 1   sd        3168 non-null   float64
 2   median    3168 non-null   float64
 3   Q25       3168 non-null   float64
 4   Q75       3168 non-null   float64
 5   IQR       3168 non-null   float64
 6   skew      3168 non-null   float64
 7   kurt      3168 non-null   float64
 8   sp.ent    3168 non-null   float64
 9   sfm       3168 non-null   float64
 10  mode      3168 non-null   float64
 11  centroid  3168 non-null   float64
 12  meanfun   3168 non-null   float64
 13  minfun    3168 non-null   float64
 14  maxfun    3168 non-null   float64
 15  meandom   3168 non-null   float64
 16  mindom    3168 non-null   float64
 17  maxdom    3168 non-null   float64
 18  dfrange   3168 non-null   float64
 19  modindx   3168 non-null   float64
 20  label     3168 non-null   obje

Все значения атрибутов числовые. Здесь преобразований не требуется

In [101]:
print(len(df[df['label']=='male']))
print(len(df[df['label']=='female']))

1584
1584


Данные сбалансированы по классам.

Уточним таргет. Теперь мальчики 1, девочки 0




In [102]:
df.loc[df['label'] == 'male', 'label'] = 1
df.loc[df['label'] == 'female', 'label'] = 0
df['label'] = df['label'].dropna().apply(lambda x: int(x))

Надо нормализовать значения

In [103]:
df_x = df.iloc[:, :-1]
scaler = MinMaxScaler()
scaler.fit(df_x)
df_norm = pd.DataFrame(data=scaler.transform(df_x), columns=df_x.columns)
df_norm['label'] = df['label']

Сэмплируем данные

In [104]:
test_df = df_norm.sample(frac=0.2, random_state=1337)
train_df = df_norm.drop(test_df.index)

print(
    f"Обучающая выборка {len(train_df)}\n"
    f"Тестовая выборка {len(test_df)}"
)

Обучающая выборка 2534
Тестовая выборка 634


In [105]:
X_train = train_df.iloc[:, :-1].values
X_test = test_df.iloc[:, :-1].values

Y_train = np.asarray(train_df['label']).astype('float32').reshape((-1,1))
Y_test = np.asarray(test_df['label']).astype('float32').reshape((-1,1))

In [111]:
model = tf.keras.Sequential()
model.add(layers.Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer=tf.keras.optimizers.Adam(0.01),
              loss='binary_crossentropy',
              metrics=['accuracy'])

callb = callbacks.EarlyStopping(
                    monitor='accuracy',
                    mode='max', min_delta=0.001,
                    patience = 5)
history = model.fit(X_train,Y_train,epochs=1000,validation_data=(X_test,Y_test),batch_size=32,
                    verbose=1)


Epoch 1/1000
80/80 [==============================] - 1s 4ms/step - loss: 0.4354 - accuracy: 0.8295 - val_loss: 0.2751 - val_accuracy: 0.9290
Epoch 2/1000
80/80 [==============================] - 0s 3ms/step - loss: 0.1885 - accuracy: 0.9337 - val_loss: 0.1357 - val_accuracy: 0.9574
Epoch 3/1000
80/80 [==============================] - 0s 3ms/step - loss: 0.1149 - accuracy: 0.9621 - val_loss: 0.1200 - val_accuracy: 0.9558
Epoch 4/1000
80/80 [==============================] - 0s 3ms/step - loss: 0.1070 - accuracy: 0.9645 - val_loss: 0.1249 - val_accuracy: 0.9511
Epoch 5/1000
80/80 [==============================] - 0s 3ms/step - loss: 0.1128 - accuracy: 0.9613 - val_loss: 0.1057 - val_accuracy: 0.9653
Epoch 6/1000
80/80 [==============================] - 0s 3ms/step - loss: 0.1014 - accuracy: 0.9665 - val_loss: 0.1037 - val_accuracy: 0.9669
Epoch 7/1000
80/80 [==============================] - 0s 3ms/step - loss: 0.0923 - accuracy: 0.9716 - val_loss: 0.1224 - val_accuracy: 0.9527
Epoch 

Нам удалось обучить простую нейронную сеть для идентификации гендера по данным информации о голосе.

Ключевые моменты, которые благоприятно повлияли на качество модели:
1. балансировка
2. нормализация данных

В качестве функции потерь выбрана - сигмоидная функция.

Модель демонстрирует высокие показатели точности. Доработка датасета на многоклассовый, где идентифицируется разные люди, позволит с высокой точностью идентифицировать людей. Возможно спасет от телефонных мошенников. По данным СМИ, многие политики попадаются на пранки. Это могло их спасти.

PS хотел сделать с изобрадениями, но интересный датасет не нашел